# lesson 6 configure azure sql db for scale and performance

**note:backup storage redundancy** 

geo redundant backup will replicate the storage to a paired region (e.g. US East to Us West) in case of data storage damaging

**note:billing type** 
- provisional charged based on vcore
- serverless charged based on second, shut down after 1 hr inactivity, restarted takes a little time


**note:hybrid benefit** 
- it's a Microsoft licensing program that allows customers to use their existing on-premises Windows Server and SQL Server licenses with active Software Assurance or subscription to reduce costs when running those workloads in Azure
- can only work with vcore, not with dtu
**note:elastic pool** 

- let manage multiple db resource allocation possible, so they could share resource when one not using it
- not possible for hyperscale
- can select edtu (elastic dtu) in setting
- unit price for edut compared to dtu is extra 50 %, vcore have same unti price
- if a DTU db peak at different time, elastic pool could be a good idea, otherwise it's more expensive due to unit price
- unavailable at hyperscale



### storage purchasing model
- can be changed later, except changed out of hyperscale
- change should be done at a low trafic time
- after change DMV(dynamic management view) needed to be flushed for accurate estimation
exec sq_query_store_flush
##### DTU (database transaction unit)based
- DTU can be adjusted with related S tier
- CDC (changed data captured) can't be used when having less than 100 DTB (= S3 = 1vcore)
- when using more than 300 DTU, vcore might be a cheaper option
- plan selection can based on peak usage and storage need, the latter could be estimate with average data use * db number 
1. Basic: up to 2GB
Development, testing, infrequently accessed workload
2. Standard
3. Premium
suitable when more iops (inputs output per second) is needed
##### vCore (virtual core) based
- max data storgae and core could be specified
- core decide max data storgae ceiling & tempdb size
- log space is 30% of the max data stoarge
- hardware choice affect memory, core number and storage
- iops = concurrent number of request, this can't be configured, but go up with vcore

4. general purpose: 80 vcore, 4 terabyte storage
suitable for low latency input
5. Hyperscale: 80 vcore, 100 terabyte storage
6. Business critical: 80 vcore, 4 terabyte storage, fast geo-recovery and advanced data corruption protection, free second read-only replica
Business critical has high transaction rate and high resiliency, is suitable for large number/long-running transaction

### table partitioning

- a table have 3 elements storage space, computing resource limit (exceeing might lead to timeout) and network bandwidth limit
- vertical scaling by adding disc space, processing power, memory and network connection could be a short term solution
- horizontal scaling by divifing the table up according to rule e.g. by year, it add up complexity but reduce hardware demand
- back up per partition is also faster 
- the other option is to split the table columns up

### database  sharding
- when we apply horizontal scaling on db level
- there would be a table with shard key, and where they are stored, this require more setting up 
- range strategy is diving by range e.g.years, sequential shard, data might be unblanced when there is seasonality
- hashing strategy introduce random element for distribution, it reduce hotspot bcs no cluster would be more used, but rebalancing and manging mmight be difficult, when looking up a value the results are all over places
- functional partitioning, not putting all the tables in the same db, e.g. seperate tables that needs extra security
- keeping data geographically close can reduce latency when using

**note: file group**
- can't be used in Azure SQL db (only primary file group there)
- file can be putted in file group to define partition range rules and where to store

### compression for table
- pro reduced space
- con takes extra power and time to compress and retrieve 
- can't be used on table with sparse columns
- changing compression require droping clustered index
- complete index viewed can be compressed
- system table can't be compressed
- table with different partition can be compressed differently
- *exec sp_estimate_data_compression_savings* help estimate compression effect

method:
1. no compression
2. row compression: char and nchar columns would be greatly reduced, varchar and int not so much, date types depends, tiny int not at all
3. page compression which includes (below element can't be activated seperately):
    1. row compression
    2. prefix compression: store long duplicated value in a specific column as prefix key, and generate an extra table for prefix key to long name to prevent the duplication of long char storage
    3. dictionary compression: similar to prefix compression, but instead of on a specific column, the duplication would be searched across columns and keys would be used for whole table

    - page is a set of row up to 8192 character
    - uncompressed at first, when additional rows can't be fit in, compression takes place
    - some older version SQL server on VM is this unavailable

**note heap**
haep is table without clustered index

**columnstore table**
above are row stored table. in columnstore table, columns are always compress, which could be further compressed through columnstore archival compression (but this has super high compute cost for uncompress)

### SQL data sync
- works for Azure SQL, on-prem or VM, but not with management instance
- allows sync across db
- primary key is required
- must define a hub db and multiple member dbs
- rules can be defined which direction to sync, e.g. change would be replicated from hub to member or member to hub, but not directly between members
- rule when hub and member simultaneously changed should be defined
- sync metadata db is an azure sql db whihc loacte in the same region as hub and store, should be empty at start
- for on perm or vm member db, a syn agent programm is needed 
- on azure db page syncing db could be created


# lesson 7 strategy of migration 

### migration tools

| purpose    | tool | extra function |
| -------- | ------- | ------- |
| lift & shift sql server to vm   | azure migrate    |   assess sql data estate at scale(across data center)/getting azure sql deployment recommendation, target sizing & monthly estimate  |
| non sql objects (DB2, MYSQL, Oracle, SAP, ASE) to sql/azure sql, each are independent download program | sql server migration system     ||
| sql server object to sql db/managed instance/vm/on prem     | data migration system    |upgrade SQL/assess sql data estate at scale/ performance & reliability improvement recommendation/ detect incompatibility of target/ migrate on-prime/ discover new feature on target|
| compare workloads between source and target  | database experiementation assistant    | |
| open source db (mysql, postgresql, mariadb) to azure offline/online (premium price tier)  | azure db migration service    ||

1. azure migrate  
- consists of 2 tools: discovery & assessment, server migration
**azure migrant hub** includes azure migration assistant & azure db migration service

2. sql server migration system 

3. data migration system
- for large size data azure db migration service  would be better option

4. database experiementation assistant 

5. azure db migration service 
- princing tier standard provides up to 4 vcore for free, but only support offline migration
- princing tier premium provides up to 4 vcore with 6 month free trial, support offline & online

| from    | to | online (continuous sync)    | offline (one-time) |
| -------- | ------- | -------- | ------- 
| SQL server  | azure sql db    | x  | v    |
| SQL server    | azure sql db mi    | v | v    |
| SQL server    | azure sql vm  | x  | v    |
| mongodb    | azure cosmos db  | v | v    |
| mySQL    | azure db for mySQL  | x  | v   |
| postgreSQL    | azure db for postgresql  | v  | x  |




**reminder** if not latest SQL is needed, then VM must be used and a window version needed to be decide

**note** 
- without downtime allowance, the only way is to do an online migration
- if cross db dependecy exist, azure db is not a good option, as it doesn't allow cross db queries
- for azure db migration service, allow outbond port 443 = port for https or 1434 used for UDP, then enable TCP IP protocal, next create azure SQL db instance which needs a server level firewall rule on depature db/server & contorl server permission on depature server to allow access to DMS and on target db control db permission is needed. it use exsiting full log backup instead of creating new one
- there are other service e.g. Bulk copie programm, command bulk insert, log data from Azure blob storage, SQL server integration service package, spark/azure factory
- migrant stuff directly between Azure SQL db can use export or export data tier application (which allow export of schema), or export form azure portal, download and using SQLpackage
- export data tier application (DAC) will make the export as .bacpac (backup package) file

### post migration
1. have tests to make sure data are there
2. checking Query returns same result in sourece and target db
3. checking the performance
4. changing the app connecting to db from departure db to target db
5. checking missin features

### lesson 8 Microsoft Entra ID


### authn = authentication = log in
1. window authentication
    - on a window machine/domain 
    - in azure only on VM is window available
2. SQL sever authentication
    - using username and password
3. azure active directory
    - setting could be access through azure portal, e.g. adding user
    - more secure than SQL sever authentication and can select way to authenticate in portal 
    - once the AAD admin is setted through azure portal (only 1 admin allow), the admin can log in SSMS through (other log in method won't allow role creation) and create more roles through query 
    - comparison between AAD admin and sql server admin:
        - both could create users based on SQL server authentication log in
        - both can create contained DB users based on SQL server authentication wo log in
        - AAD admin can create contained DB users based on aad users and group
    - authentication methods includes FIDO secutity keys which is a hardware on computer, microsoft authenticator, text message and temporary access pass
    - can sync with on-prem windows server active directory
    - when we are logging in azure portal is a form of azure active directory
    - azure active directory - integrated/universal with MFA(multi factor authentication) can be used when using admin tool like SSMS on a computer that is not domain joined 
    - 3 different way to authenticate in:
        1. cloud only identity
            - handle sign in completely in cloud by azure
        2. federated authentication
            - integrate with existing federation provider
            - sign in requirement which is not native supported by Azure active directory
        3. pass-through authentication
            - when don't have a sign-in requirement not natively supported by Azure active directory
            - and want to enforce extra feature like user level active directory sign in policy
    - identity: 
        1. cloud only identity
        2. hybrid identity support both cloud authentication & pass-through authentication
        3. hybrid identity support federated authentication
    - to create a user based on AAD user use: create user [user_email] from external provider
4. window server active directory

**note:**
- authz = autherization = what one could access

### role
-  when running on an non-azure machine that is domain joined by creating a certificate and connect the app to azure data
- to see all role in azure SQL use sp_helprole
- after creating user, adding roles for the user is needed in SQL server
    - alter role [role name] add/drop member [member email]
    1. db_owner : have access to all.
    2. db_securityadmin : modify role membership for custom roles only. manage permission, can elevate own permission
    3. db_accessadmin: add & remove access to db for login & groups
    4. db_backup_operator: not applicable in azure. can backup db, mi & vm
    5. db_ddladmin: can run ddl comment (creat, alter, drop)
    6. db_data_reader: can read all data
    7. db_deny_data_reader: can't read old data
    8. db_data_writer: ability to add, delete or change data
    9. db_deny_data_writer: can't add, delete or change data
    - on-prem only, roles that exist in master db
    10. db_manager: create/delete db
    11. login_manage: create/delete login in master db = security_admin for master db
- role based access control (RBAC) in azure:
    - Access control section is called IAM, when role could be added
    - there is no access to server wide logging permission bc access to underlying server doesn't exist
    1. SQL DB contributer: allow SQL DB, but don't have access to content
    2. SQL server contributer: allow SQL server/security policy manage, but don't have access to content
    3. SQL security manager: allow SQL security policy manage, but don't have access to content

### granting permission
- granting access to particular schema/table can't be achieved through these set roles in stead grant permission command should be used
- command: authorization permission on securable::name to pricipal with grant option
- select * from sys.fn_my_permissions(null, 'DATABASE') shows all permission
1. autherization (grant, revoke, deny)
    - grant [permission_type] on object::[schema].[table] to [user_email]
    - to revoke the permission used revoke [permission_type] on object::[schema].[table] to [user_email]
    - to ban someone from table access that cannot be overwrite by grant use deny [permission_type] on object::[schema].[table] to [user_email]
2. permission 
    - table: select, insert, update, delete, control(all right to table), reference(view foreignkey), take ownership(change ownership), view change tracking, view definition (rightclick and go to script table path)
    - schema: alter(=alter/create/drop any securable)
    - function/stored procedure: alter, execute(even lack of select permission on table is not a problem, this is called ownership changing), view change tracking, view definition
3. securable (object, schema, db, server)
4. principal (login, user, role: create role [rolename], add member [useremail])
    - public is a special role that every single user of the db have this role
5. with grant option: allow the principal to share the permission later

**note:apply principal of least privilige for all securable**
a user should have least privilige user account (LUA)


### lesson 9 implement security

### transparent data encryption (tde)
- this is the security for data in transit
- it encrypts then deencrypts data at page level at rest
- encrypts at write then decrypts at read
- for azure SQL db this is set at server level, new db inheritate the rule unless recover from a copy, encryption can be turn off in azure portal / Tsql / cloudshell / restapi
- for azure sql mi this is set at instance level,  new db inheritate the rule but not system db, encryption can be turn off  in Tsql/ cloudshell/ restapi
- it use database encryption key (dek), which is a symetric key = one single key is needed for en- & decrypts
    - this could be a service managed key, protected by a TDE protector using service managed certificate 
    - this could be a bring your own key, in this case it could also be asymmetric

**note** transparent layer security (TLS), encrypts when in transit

### object level encryption (e.g. table with sensitive data)
1. always encrypted
    - always encrypted is available in azure sql db, mi, vm
    - this could be done in ssms through right click encrypt columns
    - encryption type has deterministic (encryption is the same everytime, allows for join, group by, index, distinct without decrypt) & random 
    - when the column is used as index, it can't be randomized encrypt
    - to create encryption, key vault is needed
    - after creation, next time when connecting to db in SSMS select enable alway encrypt 
    - a pricipal (e.g. user) need to be in key vault to decrypt encrypted colunm

    - to query an encrypted column in where clause, set up a parameter & enable parameterlization for always encrypted is needed, but this will still block funtion like
    - to enable partial string comparison, always encrypted need to be used with secure enclaves
        1. enable confidential computing through configured as DC series
            - strongest security isolation
            - type name is intel software guard intellegence(sgx)
            - require ms azure attestation
        2. enable secure enclaves in a standard configure db
            - type name is virtualization based security (vbs)
            - available for all ay sql, and sql 2019+
            - not available in ger india central
            - can't defend against e.g. replacing enclaves program with malware
            - creating a protected part of memory within which data is display as plain text
            - protection against operative system threat
            - standard azure protection (e.g. multifactor authentication, just in time access)
            - can't be deactivated after activation

    - permission wise there are
        1. alter any column master key
        2. alter any column encryption key
        3. view any column master/encryption key (for access, query and read encrypted data)
    - role wise there are
        1. security admin who generate column encryption key and master key, access to key and key store are required, access to db is not required
        2. db admin who manage metadata about key, access to key and key store isn't required, access to db is required
        - if these 2 role should belongs to 2 ppl, then powershell should be used
        - if these 2 role should belongs to same ppl, then powershell or ssms could be used

2. dynamic data masking
    - this could be done through azure portal 
    - in tsql: alter table [schema].[table] alter column [column] add masked with (fuction = 'default()')
    - it can be apply on specific column with desired partial masking 2025-0x-1x
    - specific users could be excluded in azure portal/tsql: grant(/revoke) unmask to [useremail] 
    - this could also be done through powershell
### key vault 
- could be created in azure sql portal
- in key vault store secrets
- key vault name need to be unique
- better store it in region near db
- pricing tier premium allow use of hardware security model (HSM). it's the same as standard  until hsm is used
- in access policy selecting unwrap key + wrap key + verify + sign (and also the default selections) is necessary to column create primary key 


| | always encrpted    | transparent data encrpytion (tde) |
| -------- | ------- |------- |
| sql server version  |  2016+   | 2008+ |
| sql server entriprise edition | x     | v|
| free in azure sql db    | v    | v|
| protect data at rest   | v   | v |
| protect data in use    | v  | x (when in transport, using transport layer security) |
| protect data from sql admin  | v  | x |
| data is de/encrypted on the client side  | v | x|
| data is de/encrypted on the server side   | x | v |
| encrypt at    | column level  | entire db|
| transparent to application   | partially | v |
| encryption option   | v | x |
| key management   | customer managed keys | server/customer managed keys |
| protects key in use  | v | x |
| driver required | v |  x|


### configure server level firewall
- by default all connection are rejected
